In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/next-word-prediction/1661-0.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Embedding,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from tensorflow.keras.utils import to_categorical

In [3]:
sent = pd.read_csv('/kaggle/input/next-word-prediction/1661-0.txt',delimiter=',', on_bad_lines='skip')

In [4]:
sent

,Project Gutenberg's The Adventures of Sherlock Holmes,by Arthur Conan Doyle
0,This eBook is for the use of anyone anywhere a...,NaN
1,almost no restrictions whatsoever. You may co...,give it away or
2,re-use it under the terms of the Project Guten...,NaN
3,with this eBook or online at www.gutenberg.net,NaN
4,Title: The Adventures of Sherlock Holmes,NaN
...,...,...
7655,Most people start at our Web site which has th...,NaN
7656,facility: www.gutenberg.org,NaN
7657,This Web site includes information about Proje...,NaN
7658,including how to make donations to the Project...,NaN


In [5]:
sent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7660 entries, 0 to 7659
Data columns (total 2 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   Project Gutenberg's The Adventures of Sherlock Holmes  7660 non-null   object
 1    by Arthur Conan Doyle                                 3039 non-null   object
dtypes: object(2)
memory usage: 119.8+ KB


In [6]:
file_path = '/kaggle/input/next-word-prediction/1661-0.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    sentence = "".join(lines)

In [7]:
type(sentence)

str

In [8]:
#removing special characters and adjusting spacing
cleaned_text = re.sub(r'[^\x00-\x7F]+', '', sentence)
cleaned_text = cleaned_text.replace('\n',' ').replace('\r','')
cleaned_text = re.sub('\s+',' ',cleaned_text).strip()
cleaned_text[:100]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the u"

In [9]:
#performing tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([sentence])
len(tokenizer.word_index)

sequence_length = 5
sequences = []
targets = []
vocabulary = set()

for sentence in sent:
    tokens = word_tokenize(sentence)
    vocabulary.update(tokens)
    
vocabulary = sorted(list(vocabulary))
word_index = {word: index for index,word in enumerate(vocabulary)}
index_word = {index: word for word,index in word_index.items()}
vocab_size = len(vocabulary)

In [11]:
faqs=cleaned_text.split('. ')

input_sequences = []
for sentence in faqs:
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [12]:
len(input_sequences)

101544

In [13]:
max_len = max([len(x) for x in input_sequences])
print(max_len)
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')
print(padded_input_sequences.shape)

105
(101544, 105)


In [14]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [15]:
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y = to_categorical(y, num_classes=8932)

# Print the shape of the one-hot encoded labels
print(y.shape)


(101544, 8932)


In [16]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

model = Sequential([
    Embedding(8932, 128, input_length=104),
    Bidirectional(LSTM(128, return_sequences=True)),  # BiLSTM layer
    LSTM(64),                                          # Additional LSTM layer
    Dense(8932, activation='softmax')
])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
model.fit(X,y,epochs=2,batch_size=64)

Epoch 1/2
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 472s 294ms/step - accuracy: 0.0556 - loss: 6.7977
Epoch 2/2
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 466s 293ms/step - accuracy: 0.0821 - loss: 6.0181


In [19]:
import time

def predict_word(text):
    for i in range(10):
      # tokenize
      token_text = tokenizer.texts_to_sequences([text])[0]
      # padding
      padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
      # predict
      pos = np.argmax(model.predict(padded_token_text))

      for word,index in tokenizer.word_index.items():
        if index == pos:
          text = text + " " + word
          print(text)
          time.sleep(2)


In [20]:
seed_text = "The Adventures is"
next_word = predict_word(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step
The Adventures is the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
The Adventures is the man
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
The Adventures is the man and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
The Adventures is the man and the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
The Adventures is the man and the man
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
The Adventures is the man and the man and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
The Adventures is the man and the man and the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
The Adventures is the man and the man and the man
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
The Adventures is the man and the man and the man and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
The Adventures is the man and the man and the man and the


In [21]:
model.save('next_word_prediction.h5')

In [22]:
model.save_weights('next_word.weights.h5')